In [ ]:
import os
import sys
import datetime
from glob import glob

import numpy as np
import matplotlib.pyplot as plt
import matplotlib

# on my computer NX2 is not installed, I just import directly from the development directory
#sys.path.append(os.path.abspath('../NX2/'))
sys.path.append('/Users/hamogu/code/NX2/')
import NX2
import NX2.polar
import NX2.math
sys.path.append(os.path.abspath('.'))
import utils

%matplotlib inline

ENTWURF = False

In [ ]:
datapath = '/Users/hamogu/MITDropbox/NX2/'
#datapath = '/melkor/d1/guenther/Dropbox/NX2/'
plotpath = os.path.join(datapath, '2018/')
#geojsonpath = '/melkor/d1/guenther/projects/NX2/geojson/'
geojsonpath = '/Users/hamogu/projects/NX2/geojson/'

In [ ]:
# Move this into utils files once it's ready to be canned
filelist = glob(os.path.join(datapath, '2018', '2018*csv'))
filelist.sort()

In [ ]:
data = [NX2.NX2Table(f, utils.date_from_filename(f), remove_empty=False) for f in filelist]
# Compass data and all related columns are empty in first file (and thus deleted on read-in)
#for n in ['HDC', 'DFT', 'SET']:
#    data[0].add_column(n, np.nan)
# We don't have a rowfile with that info yet
for d in data:
    d.add_column('sailing', 0)

In [ ]:
# Could use atpy to read in table, but I'm more used to astropy, so this is just faster
from astropy.table import Table
sailing = Table.read(os.path.join(datapath, '2018', 'sailing.csv'), format='ascii')

In [ ]:
for d in data:
    d.add_empty_column('rowpermin', np.int_)
    for row in sailing:
        year, month, day = row['date'].split(':')
        t1 = tuple([int(x) for x in row['setzen'].split(':')])
        t2 = tuple([int(x) for x in row['hoch'].split(':')])
        inddate = (d.year == int(year)) & (d.month == int(month)) & (d.day == int(day))
        indtime = (d.time() >= datetime.time(*t1)) & (d.time() <= datetime.time(*t2))
        d['sailing'][inddate & indtime] = 1        
        d['rowpermin'][inddate & indtime] = 99 * row['riemen']

In [ ]:
from copy import deepcopy
merged = deepcopy(data[0])
for d in data[1:]:
    merged.append(d)

## Bestimmung des Korrekturfaktors

In [ ]:
def fitplot_BSP(data, ax, titel = ''):
    fit, ind = data.fit_BSP_corr()
    ax.plot(data.SOG[~ind], data.BSP[~ind], 'y.', label = 'data not used for fit')
    ax.plot(data.SOG[ind], data.BSP[ind],'k.', label = 'data')
    ax.plot(plt.xlim(), fit.beta * plt.xlim(), label = 'fit')
    ax.set_title(titel + '$\\beta = ${0:4.2f}'.format(fit.beta[0]))

In [ ]:
fig = plt.figure(figsize = (20,10))
for i, d in enumerate(data):
    ax = fig.add_subplot(4,4,i+1)
    fitplot_BSP(d, ax, titel = str(i)+': ')
    ax.set_xlim([0,5])
    ax.set_ylim([0,5])
    if i in [0,4,8,12]:
        ax.set_ylabel('BSP')
    if i in [12,13,14,15]:
        ax.set_xlabel('SOG')

Jede Grafik hier zeigt einen Datenssatz. Die hellen Punkte sind alle Datenpunkte in diesem Datensatz, die dunkel hervorgehobenen sind die Punkte, die geeignet sind, den Korrekturfaktor fur das Log zu bestimmen.

In einigen Fahrten fehlte der Kompass, weil das Kabel defekt war. Fur diese Fahrten konnen wir keinen Korrekturfaktor mit unserem normalen Algorithmus bestimmen, weil wir dazu den Kompasskurs mit den GPS-Kurs vergleichen und nur Punkte verwenden, bei denen diese beiden relativ gut ubereinstimmen. Dies ist notwending, um zu verhindern, dass Strecken, bei denen das Schiff seitwarts driftet, den Wert verfalschen. 

Die Steigungen der Regressionsgeraden liegen im Bereich 0.8-0.9. Auch wenn wir hier keine formalen Unsicherheiten auf diesen Messwert angeben, so kann man doch mit dem Auge erkennen, dass immer einige dunkle Punkte von der Gerade abweichen und gerade fur kurzere Fahrten kann das den gemessenen Korrekturfaktor beeinflussen. Das kann zum Beispiel gesschen, wenn der Wind with mit der Zeit andert. 

Die Halterung des Log wurde mindestens zwei Mal durch Grundkontakt beschadigt (am zweiten und am dritten Tag der Messfahrten), aber wir sehen keine signifikatne Anderung des Korrekturfaktors. Offensichtlich wurde das Log also in gleicher Position wieder eingebaut.

Mit Blick auf die oben gezeigten Regressionen, verwenden wir den Korrekturfaktor 0.85.

In [ ]:
for d in data:
    d.BSP = d.BSP / 0.85

In [ ]:
for d in data:
    d.plot_course(scale=20, n=60)

In [ ]:
for i, d in enumerate(data):
    d.write_geojson(os.path.join(geojsonpath, '2018', os.path.basename(filelist[i]) + '.geojson'))

In [ ]:
fig = plt.figure(figsize=(12, 12))
for i, d in enumerate(data):
    name = os.path.basename(filelist[i][:-7])
    ax = fig.add_subplot(4, 3, i + 1)
    ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%H:%M', tz=None))
    sog = ax.plot(d.datetime(), d.SOG, label='GPS')
    bsp = ax.plot(d.datetime(), d.BSP, label='Log - orginal')
    xlab = ax.get_xticklabels()
    for label in xlab: label.set_rotation(30)
    ax.set_xlabel('Uhrzeit')
    ax.set_ylabel('Geschwindigkeit [Knoten]')
    ax.set_title(name)
    #ax.legend()
fig.subplots_adjust(hspace=0.5)

In [ ]:
import mpld3

fig = data[9].plot_speeds()
mpld3.save_html(fig, '../test2018.html')
mpld3.display(fig)

In [ ]:
for i, d in enumerate(data):
    name = os.path.basename(filelist[i][:-7])
    fig = plt.figure(figsize=(12, 4))
    ax = fig.add_subplot(111)
    ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%H:%M', tz=None))
    sog = ax.plot(d.datetime(), d.SOG, label='GPS')
    bsp = ax.plot(d.datetime(), d.BSP, label='Log - orginal')
    xlab = ax.get_xticklabels()
    for label in xlab: label.set_rotation(30)
    ax.set_xlabel('Uhrzeit')
    ax.set_ylabel('Geschwindigkeit [Knoten]')
    ax.set_title(name)
    ax.legend()
    mpld3.save_html(fig, os.path.join(datapath, '2018', 'figures', name + '.html'))


## Polardiagramme

### Vorbereitendes Datenfiltern

Bevor wir die Daten zu Polardiagrammen zusammenstellen, müssen wir sie filtern, denn nicht jeder Datenpunkt kann dafür genutzt werden. In diesem Abschnitt untersuchen wir, welche Filter sinnvoll sind. Die verschiedenen Filter, die wir auf die Daten anwenden, sollen verschiedene Effekte berücksichtigen: Zuerst beschränken wir uns auf die Messpunkte, die beim Segeln aufgenommen wurden (ohne Hilfe der Ruderer). Dann verlangen wir, dass die Geschwindikgeit sich nur langsam ändert, denn wenn z.B. der Bug in den Wind gedreht wird, dann hat das Schiff zunächst noch eine Geschwindigkeit, die aber schnell abnimmt. Das bedeutet nicht, das wir gegen den Wind segeln können, sondern nur dass die Daten, die direkt nach einer Richtungsänderung, direkt nach dem Hissen des Segels oder in einer Windböhe aufgenommen wurden, herausgefiltert werden müssen. Außerdem ändert sich der Wind sehr viel schneller, als das Schiff seine Geschwindigkeit anpassen kann. Deshalb glätten wir die Windgeschwindigkeit etwas, um diesen Effekt auszugleichen. Dafür wählen wir eine Funktion, die nur vorausgegangene, aber nicht zukünftige Messwerte verwendet; deshalb erscheinen die Maxima im geglätteten Wind immer etwas später. Die nächste Grafik zeigt ein Beispiel dafür.

In [ ]:
fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)
ax.plot(merged['TWS'][3600:3800], label = 'gemessener Wind')
ax.plot(NX2.math.smooth_expdec(merged['TWS'], 10)[3600:3800], label = u'gegl\u00e4tteter Wind')
ax.set_xlabel('Zeit [Sekunden]')
ax.set_ylabel('Windgeschwindigkeit [Knoten]')
ax.legend()
ax.set_ylim([0,10.9])

In der nachsten Grafik schauen wir uns den Winkel zum Wind an. Dabei bedeutet $0^{\circ}$, das die Galeere "im Wind" steht (der Bug in den Wind zeigt), während $180^{\circ}$ meint, das der Wind von achtern kommt und die Galeere "vor dem Wind" segelt. Das Vorzeichen kann dabei positiv oder negativ sein, je nachdem ob der Wind von Steuerbord oder von Backbord kommt. 
Durch diese Skala kann es bei achterlichem Wind zu scheinbaren Spüngen kommen (z. B. um Sekunde 15 herum in der folgenden Grafik), wenn der Wind von $+179^{\circ}$ zu $-179^{\circ}$ wechselt. Da wir in der folgenden Analyse nicht zwischen Backbord und Steuerbord unterscheiden, genügt es, den Betrag des Windwinkels zu betrachten und so das Problem zu vermeiden. Außerdem sieht man in der Grafik, dass die gemessene Windrichtung sehr schnell hin- und herspringt, weil der Windmesser im Wind flattert. Dies ist besonders bei schwachen Wind der Fall und bei Wind vom Bug, wenn das Segel den Windmesser abdeckt. Der Windmesser ist zwar mit einem "twin fin" System ausgestattet, dass ein Hin-und Herflattern vermeiden soll, aber die gemessene Windrichtung schwankt immer noch im Sekundenbereich. Wenn wir dies nicht berücksichtigen, dann werden bei der Konstruktion eines Polardiagrams viele Messwerte bei unpassenden Windwinkeln einsortiert. 
Deshalb glätten wir auch die Windrichtung (z. B. schwankt der Windwinkel in der folgenden Grafik kurz vor Sekunde 150 von fast 0 auf 150, während sich die Geschwindigkeit in dieser kurzen Zeit kaum verändert.).

In [ ]:
fig = plt.figure(figsize=(4,4))
ax = fig.add_subplot(111)
ax.plot(merged['TWA'][3600:3800], label = 'Windwinkel')
ax.plot(np.abs(merged['TWA'][3600:3800]), label = "Betrag des Winkels")
ax.plot(NX2.math.smooth_expdec(np.abs(merged['TWA'][3600:3800]), 10), label = u"Winkel geglättet")
ax.set_xlabel('Zeit [Sekunden]')
ax.set_ylabel('Winkel zum Wind [Grad]')
ax.legend(loc = 'lower left')


In [ ]:
sail  = NX2.polar.sail(merged)
con = NX2.polar.near_const(NX2.math.smooth_gauss(merged['BSP'],10), max_diff = 0.007)
TWSs = NX2.math.smooth_expdec(merged['TWS'], 10)
TWAs = NX2.math.smooth_expdec(np.abs(merged['TWA']), 10)
conTWA = NX2.polar.near_const(TWAs, max_diff = 1.5)

In [ ]:
print 'Anzahl Datenpunkte mit Segel und ohne Ruder', sail.sum()
print u'zusätzliche Bedingung: Fast konstante Geschwindigkeit', (sail & con).sum()
print u'zusätzliche Bedingung: Fast konstanter Windwinkel', (sail & con & conTWA).sum()

In [ ]:
fig = plt.figure(figsize = (10,5))
NX2.polar.grid(merged.TWA[sail], merged.TWS[sail], merged.BSP[sail], 
               np.arange(0.,12.1,4.), np.arange(0.,181., 30.1), fig = fig)

Man sieht schon, dass (außer in der letzten Zeile, wo wir einfach sehr wenige Daten haben) die meisten Verteilungen sehr gut um einen Mittelwert herum liegen. Allerdings sind manche Verteilungen noch so breit, dass man sich fragt, ob wir da vielleicht noch etwas mehr weglassen müssen. In der nächsten Grafik verweden wir nun die geglätteten Werte für Windgeschwindikeit und -winkel. Außerdem filtern wir die Zeiten heraus, bei denen sich die Bootsgeschwindigkeit oder der Windwinkel schnell ändern. Dabei ist es schwer, genau zu definieren, an welcher Stelle man hier abschneiden soll. Ich habe die Werte so gewählt, das nur die wirklich extremen Werte (wenn zum Beispiel eine enge Kurve gesegelt wird) herausgenommen werden. 

In [ ]:
fig = plt.figure(figsize = (10,5))
ind = sail & con & conTWA
NX2.polar.grid(TWAs[ind], TWSs[ind], merged.BSP[ind], np.arange(0.,12.1,4.), np.arange(0.,181., 30.1), fig = fig)

Die Grafik sieht im wesentlichen noch so ähnlich aus wie darüber. Einige der Verteilungen sind etwas schmaler geworden, aber fur kleine Windgeschwindigkeiten gibt es immer noch Daten, die anzeigen, dass das Schiff gegen den Wind gesegelt ist. 

In [ ]:
def my_polar(data, drift=False, rawTWA=False, fct=np.median, 
             anglebins=np.arange(0, 181., 15.01), speedbins=np.arange(1.,16.,3.)):
    '''Group data into arrays for polar diagrams

    Parameters
    ----------
    data : NX2 data table
    drift : bool
        If true, return drift angle instead of BSP
    rawTWA : bool
        Usually, the grouping is done by the drift corrected TWA, i.e. the direction the ship actually moves in.
        If True, the uncorrected TWA is used.
    fct : function
        see documentation of NX2.polar.group
    '''
    con = NX2.polar.near_const(NX2.math.smooth_gauss(data['BSP'],10), max_diff = 0.007)
    TWSs = NX2.math.smooth_expdec(data['TWS'], 10)
    TWAs = NX2.math.smooth_expdec(np.abs(data['TWA']), 10)
    conTWA = NX2.polar.near_const(TWAs, max_diff = 1.5)
    TWAsdrift = np.abs(TWAs)+np.abs(NX2.math.bearingdiff180(data.HDC, data.COG))
    ind = con & conTWA & NX2.polar.sail(data)
    if not drift:
        if rawTWA:
            return NX2.polar.group(TWAs[ind], TWSs[ind], data['BSP'][ind], speedbins, anglebins, fct=fct)
        else:
            return NX2.polar.group(TWAsdrift[ind], TWSs[ind], data['BSP'][ind], speedbins, anglebins, fct=fct)
    else:
        DFT= np.abs(NX2.math.bearingdiff180(data['HDC'], data['COG']))
        DFTs = NX2.math.smooth_expdec(DFT, 10)
        return NX2.polar.group(TWAs[ind], TWSs[ind], DFTs[ind], speedbins, anglebins, fct=fct)

In [ ]:
anglebins=np.arange(0, 181., 15.01)
speedbins=np.arange(1.,11,2.)

In [ ]:
fig = plt.figure(figsize = (10,5))
aux1, ax1 = NX2.polar.setup_plot(fig, 121, maxr=4)
aux2, ax2 = NX2.polar.setup_plot(fig, 122, maxr=50)

colors = ['r','g','b','k','y','c','orange','r']
poldat = my_polar(merged, drift=False, fct=np.mean, speedbins=speedbins, anglebins=anglebins)
temp = NX2.polar.plot(aux1, poldat, speedbins, anglebins, color = colors)
ax1.set_title('Polardiagram')

poldat = my_polar(merged, drift=True, fct=np.mean, speedbins=speedbins, anglebins=anglebins)
temp = NX2.polar.plot(aux2, poldat, speedbins, anglebins, color = colors)
ax2.set_title('Polardiagram - Drift')

ax2.axis["left"].label.set_text(r"Driftwinkel [$^{\circ}$]")